<a href="https://colab.research.google.com/github/polydeuces32/-TensorTrades-AI-Powered-Trading-Bot-for-SPY/blob/main/TensorTrades_(SPY0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reinstall required libraries after reset
!pip install yfinance tensorflow scikit-learn matplotlib ipywidgets --quiet


In [ ]:
# Revised Version: Full Pipeline + Sharpe Ratio Advice Button (via Interactive Widgets for Jupyter/Colab)

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization

# Load SPY data
data = yf.download('SPY', start='2018-01-01', interval='1d')[['Close']]
data['Close'] = data['Close'].ffill()
data['Volatility'] = data['Close'].rolling(window=10).std()
data.dropna(inplace=True)

# Normalize price
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Close']].values)
sequence_length = 60

# Prepare LSTM sequences
X, y = [], []
for i in range(sequence_length, len(scaled_data) - 1):
    X.append(scaled_data[i-sequence_length:i, 0])
    y.append(scaled_data[i+1, 0])
X, y = np.array(X), np.array(y)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Build and train the model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(X.shape[1], 1))),
    Dropout(0.3), BatchNormalization(),
    LSTM(64, return_sequences=True),
    Dropout(0.3), BatchNormalization(),
    LSTM(32), Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32, verbose=0)

# Backtest (Long + Short)
initial_balance = 10000
balance = initial_balance
positions, returns = [], []
close_prices = data['Close'].values
volatilities = data['Volatility'].values
scaled_close = scaled_data.flatten()

for i in range(sequence_length, len(close_prices) - 1):
    seq = scaled_close[i-sequence_length:i].reshape(1, sequence_length, 1)
    pred_scaled = model.predict(seq, verbose=0)
    predicted_price = scaler.inverse_transform(pred_scaled)[0][0]

    entry = close_prices[i]
    next_day = close_prices[i+1]
    vol = volatilities[i]
    shares = balance / entry
    tp = entry * (1 + 1.5 * vol / entry)
    sl = entry * (1 - 1.0 * vol / entry)

    if predicted_price > entry:
        exit = tp if next_day >= tp else sl if next_day <= sl else next_day
        pnl = (exit - entry) * shares
        positions.append(("Long", entry, exit, pnl))
    elif predicted_price < entry:
        exit = sl if next_day <= sl else tp if next_day >= tp else next_day
        pnl = (entry - exit) * shares
        positions.append(("Short", entry, exit, pnl))
    else:
        continue

    balance += pnl
    returns.append(float(balance))

# Performance metrics
returns_series = pd.Series([float(p[3]) for p in positions]).cumsum() + initial_balance
daily_returns = returns_series.pct_change().dropna()
sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252) if not daily_returns.empty else 0.0
rolling_max = returns_series.cummax()
drawdown = (returns_series - rolling_max) / rolling_max
max_drawdown = drawdown.min()

# Plot Equity Curve
trade_dates = pd.date_range(start=data.index[sequence_length], periods=len(returns_series))
plt.figure(figsize=(14, 6))
plt.plot(trade_dates, returns_series.values, label="Portfolio Value", linewidth=2)
plt.title("📈 TensorTrades Equity Curve", fontsize=16)
plt.xlabel("Date")
plt.ylabel("Portfolio Value ($)")
plt.grid(True)
plt.legend()
plt.show()

# Plot Drawdown Curve
plt.figure(figsize=(14, 4))
plt.plot(trade_dates, drawdown.values, label="Drawdown", color='red')
plt.title("📉 Drawdown Over Time", fontsize=14)
plt.xlabel("Date")
plt.ylabel("Drawdown %")
plt.grid(True)
plt.legend()
plt.show()

# Show final metrics
summary_text = f"""
### 📊 Performance Summary
- Initial Investment: ${initial_balance:.2f}
- Final Balance: ${float(balance):.2f}
- Total Profit/Loss: ${float(balance - initial_balance):.2f}
- Sharpe Ratio: {sharpe_ratio:.3f}
- Max Drawdown: {max_drawdown:.2%}
"""

display(Markdown(summary_text))

# Button to display improvement advice
def show_advice(b):
    advice_md = """
### 🧠 Sharpe Ratio Improvement Suggestions:
- ✅ Filter out low-confidence predictions
- ✅ Add multi-day prediction horizon (e.g. 3-5 days)
- ✅ Use tighter stop-loss settings to limit risk
- ✅ Ensemble model integration (LSTM + GRU)
- ✅ Benchmark against SPY Buy & Hold
"""
    display(Markdown(advice_md))

improve_button = widgets.Button(description="📐 Improve Sharpe Ratio")
improve_button.on_click(show_advice)
display(improve_button)

# Display last 5 trades
pd.DataFrame(positions[-5:], columns=["Type", "Entry", "Exit", "PnL"])
